# Reading histograms

One of the most common tasks will be translating [ROOT](https://root.cern.ch) histograms into the HEPData format. `hepdata_lib` will help you with that, and this notebook will demonstrate how to do that.

As explained in the [Getting started notebook](Getting_started.ipynb), a `Submission` needs to exist or be created. Here, we'll just create one without any additional information:

In [51]:
from hepdata_lib import Submission
submission = Submission()

The plot will be a `Table`, in this example Figure 4a from page 12 (upper left) of the [publication](https://cms-results.web.cern.ch/cms-results/public-results/publications/B2G-17-009/index.html), which shows the distribution of the reconstructed B quark mass for the data as well as the individual simulated processes. Let's add all this, some more details as well as the actual plot (for thumbnail creation) to the `Table`:

In [52]:
# Submission text
submission.read_abstract("abstract.txt")
submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/SUS-21-002/")
#submission.add_link("arXiv", "Fill later")
#submission.add_record_id(1111, "inspire")

In [53]:
from hepdata_lib import Table
# table = Table("Figure 4a")
# table.description = "Distribution in the reconstructed B quark mass, after applying all selections to events with no forward jet, compared to the background distributions estimated before fitting. The plot refers to the low-mass mB analysis. The expectations for signal MC events are given by the blue histogram lines. Different contributions to background are indicated by the colour-filled histograms. The grey-hatched error band shows total uncertainties in the background expectation. The ratio of observations to background expectations is given in the lower panel, together with the total uncertainties prior to fitting, indicated by the grey-hatched band."
# table.location = "Data from Figure 4 (upper left), located on page 12."
# table.keywords["observables"] = ["N"]
# table.add_image("example_inputs/CMS-B2G-17-009_Figure_004-a.pdf")

table = Table("Figure 3a")
table.description = "SM background prediction vs. observation in the b-veto signal region"
table.location = "Data from Figure 3 (upper left), located on page 13."
table.keywords["cmenergies"] = ["13000"]
table.keywords["observables"] = ["N"]
table.add_image("CMS-SUS-21-002_Figure_003-a.pdf")
table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
#table.keywords["reactions"] = ["P P --> GRAVITON --> W+ W-", "P P --> WPRIME --> W+/W- Z0"]

The individual plot components are stored in different ROOT files, one for the individual background processes (one histogram per process plus the total), another one for the data, and the third for the signal process. All histograms here are of type [TH1](https://root.cern.ch/doc/master/classTH1.html), but you can also read in 2-dimensional [TH2](https://root.cern.ch/doc/master/classTH2.html) using `read_hist_2d(...)` instead of `read_hist_1d(...)`:

In [54]:
from hepdata_lib import RootFileReader

# reader = RootFileReader("example_inputs/mlfit_lm_1000.root")
# reader_data = RootFileReader("example_inputs/Data_cat0_singleH.root")
# reader_signal = RootFileReader("example_inputs/BprimeBToHB1000_cat0_singleH.root")

# TotalBackground = reader.read_hist_1d("shapes_prefit/cat0_singleH/total_background")
# TT = reader.read_hist_1d("shapes_prefit/cat0_singleH/TT")
# QCD = reader.read_hist_1d("shapes_prefit/cat0_singleH/QCDTT")
# WJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/WJets")
# ZJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/ZJets")

# Data = reader_data.read_hist_1d("h_bprimemass_SRlm")

# signal = reader_signal.read_hist_1d("h_bprimemass_SRlm")

reader = RootFileReader("combined_PostFitPredictions_Run2_Mask.root")
#reader_signalWZ = RootFileReader("onlySR_TChiWZ_600_1_MCRun2.root")
#reader_signalWW = RootFileReader("onlySR_TChiWW_500_1_MCRun2.root")
reader_signalWZ = RootFileReader("onlySR_WZrun2_600_1.root")
reader_signalWW = RootFileReader("onlySR_WWrun2_500_1.root")

TotalBackground = reader.read_hist_1d("TotalBG")
ZEROONERES = reader.read_hist_1d("zeroOneres")
TWORES = reader.read_hist_1d("Twores")

Data = reader.read_hist_1d("DataFit")

signalWZ = reader_signalWZ.read_hist_1d("METvBin")
signalWW = reader_signalWW.read_hist_1d("METvBin")

The content of the histograms is stored as a dictionary, with keys `x` (bin center), `y` (bin value or for `TH2` the bin center of the 2nd dimension), `z` (`TH2` only: bin value), as well as the bin errors `dy` (`dz` for `TH2`). Furthermore, the lower and upper bin edges (`x_edges`, for `TH2` also `y_edges`) are stored for each bin:

In [55]:
TotalBackground.keys()

dict_keys(['x', 'y', 'x_edges', 'x_labels', 'dy'])

The `RootFileReader` automatically recognises if the histogram has symmetric or assymmetric errors based on [TH1::GetBinErrorOption()](https://root.cern.ch/doc/master/classTH1.html#ac6e38c12259ab72c0d574614ee5a61c7). Symmetric errors are returned if this returns `TH1::kNormal`, in this case (as for the example here) the errors are a plain `float` per bin, otherwise a `tuple` of `float`. The bin edges are always stored as `tuple`:

In [56]:
from __future__ import print_function
for key in TotalBackground.keys():
    print(key, type(TotalBackground[key]), type(TotalBackground[key][0]))

x <class 'list'> <class 'float'>
y <class 'list'> <class 'float'>
x_edges <class 'list'> <class 'tuple'>
x_labels <class 'list'> <class 'str'>
dy <class 'list'> <class 'float'>


Now define the variables:

In [57]:
from hepdata_lib import Variable, Uncertainty

#-----------

# x-axis: MET
#print(Data["x_edges"])
xBins = Variable("$p_{miss}^{T}$", is_independent=True, is_binned=True, units="GeV")
# temp = []
# for i in range(0,len(Data["x"])-1):
#     temp.append((Data["x"][i],Data["x"][i+1]))
# xBins.values = temp
xBins.values = Data["x_edges"]

sigWZ = Variable("Number of WZ signal events", is_independent=False, is_binned=False, units="")
sigWZ.values = signalWZ["y"]

# y-axis: N events
sigWW = Variable("Number of WW signal events", is_independent=False, is_binned=False, units="")
sigWW.values = signalWW["y"]

totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
totalbackground.values = TotalBackground["y"]

zeroOneRes = Variable("Number of 0,1 res BG events", is_independent=False, is_binned=False, units="")
zeroOneRes.values = ZEROONERES["y"]

twoRes = Variable("Number of rare BG events", is_independent=False, is_binned=False, units="")
twoRes.values = TWORES["y"]

data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
data.values = Data["y"]

For the data as well as the background total, we will also provide the associated uncertainties:

In [58]:
from hepdata_lib import Uncertainty

unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=True)
unc_totalbackground.values = TotalBackground["dy"]

unc_data = Uncertainty("Data uncertainty", is_symmetric=True)
unc_data.values = Data["dy"]

totalbackground.add_uncertainty(unc_totalbackground)
data.add_uncertainty(unc_data)

Now we can add the variables to the `Table` and the `Table` to the `Submission`, and create the files. Please refer to the [Getting started notebook](Getting_started.ipynb) for a complete example.

In [59]:
#table.add_variable(mmed)
table.add_variable(xBins)
table.add_variable(zeroOneRes)
table.add_variable(twoRes)
table.add_variable(totalbackground)
table.add_variable(sigWZ)
table.add_variable(sigWW)
table.add_variable(data)

submission.add_table(table)

#submission.create_files("submission",remove_old=True)

In [60]:
#!head submission/figure_3a.yaml

In [61]:
# ratio plots
def data_pred_ratio(inputfile, sig, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)
    
    Data = reader.read_hist_1d("DataFit")
    TotalBackground = reader.read_hist_1d("TotalBG")
    
#     data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
#     data.values = Data["y"]
    
#     totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
#     totalbackground.values = TotalBackground["y"]

    xBins = Variable("$p_{miss}^{T}$", is_independent=True, is_binned=True, units="GeV")
    xBins.values = Data["x_edges"]

    ratio = Variable("N_{obs}/N_{exp}", is_independent=False, is_binned=False, units="")
    ratio.values = [d/b for d, b in zip(Data["y"], TotalBackground["y"])]
    
    data_unc = Uncertainty("Data uncertainty", is_symmetric=True)
    data_unc.values = [du/b for du, b in zip(Data["dy"], TotalBackground["y"])]
    
    central = Variable("Background uncertainty", is_independent=False, is_binned=False, units="")
    central.values = [1.0] * len(Data["x"])
    

    
    if sig == "METvBin_FBWH_SR":
        central_unc = Uncertainty("Bkg. uncertainty", is_symmetric=False)        
        uncpos = TotalBackground["dy"][:-2]+[1.12,1.09]
        uncneg = [i*-1 for i in TotalBackground["dy"][:-2]]+[-0.5,-0.43]
                
        uncP = [bu/b for bu, b in zip(uncpos, TotalBackground["y"])]
        uncM = [bu/b for bu, b in zip(uncneg, TotalBackground["y"])]
        
        central_unc.values = list(zip(uncM,uncP))
        
    elif sig == "METvBin_FBW_SR":
        central_unc = Uncertainty("Bkg. uncertainty", is_symmetric=False)
        uncpos = TotalBackground["dy"][:-1]+[1.70]
        uncneg = [i*-1 for i in TotalBackground["dy"][:-1]]+[-1.03]
        
        uncP = [bu/b for bu, b in zip(uncpos, TotalBackground["y"])]
        uncM = [bu/b for bu, b in zip(uncneg, TotalBackground["y"])]
       
        central_unc.values = list(zip(uncM,uncP))
        
    else:       
        central_unc = Uncertainty("Bkg. uncertainty", is_symmetric=True)
        central_unc.values = [bu/b for bu, b in zip(TotalBackground["dy"], TotalBackground["y"])]
    
    print(f'{central_unc.values=}')
    
    ratio.add_uncertainty(data_unc)
    central.add_uncertainty(central_unc)
    
    table.add_variable(xBins)
    table.add_variable(ratio)
    table.add_variable(central)

    submission.add_table(table)
    

In [62]:
#data_pred_ratio(inputfile, sig, tablename, desc, loc, kw, addImage)
data_pred_ratio("combined_PostFitPredictions_Run2_Mask.root", "METvBin", "Figure 3a ratios", \
 "SM background observation/prediction in the bVeto signal region", \
 "Data from Figure 3 (upper left), located on page 13.", ["N/N"], "Figure_003-a.pdf")


central_unc.values=[0.10478492218085124, 0.13990525673903845, 0.17382612807511974, 0.2407982059624376, 0.25729148341104563, 0.26999258458007197, 0.2877054063694887, 0.3600874903007981, 0.8471982007431388]


In [63]:
#b tag met plots
def met_plot_btag(inputfile, sig, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)
    
    reader_signalWZ = RootFileReader("onlySR_TChiWZ_800_100_MCRun2.root")
    reader_signalWW = RootFileReader("onlySR_TChiWH_1000_100_MCRun2.root")
    
    TotalBackground = reader.read_hist_1d("TotalBG")
    TOP = reader.read_hist_1d("Top")
    NONRES = reader.read_hist_1d("Nonres")
    RARE = reader.read_hist_1d("Rare")

    Data = reader.read_hist_1d("DataFit")

    signalWZ = reader_signalWZ.read_hist_1d(sig)
    signalWH = reader_signalWW.read_hist_1d(sig)
    
    xBins = Variable("$p_{miss}^{T}$", is_independent=True, is_binned=True, units="GeV")
    xBins.values = Data["x_edges"]

    sigWZ = Variable("Number of WZ signal events", is_independent=False, is_binned=False, units="")
    sigWZ.values = signalWZ["y"]

    sigWH = Variable("Number of WH signal events", is_independent=False, is_binned=False, units="")
    sigWH.values = signalWH["y"]

    totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
    totalbackground.values = TotalBackground["y"]

    #print(f'{totalbackground.values=}')
    
    top = Variable("Number of top BG events", is_independent=False, is_binned=False, units="")
    top.values = TOP["y"]
    if sig == "METvBin_FBWH_SR":
        top.values[-2] = 0
        top.values[-1] = 0
        
    if sig == "METvBin_FBW_SR":   
        top.values[-1] = 0
        
    #print(top.values)

    nonres = Variable("Number of 0-res BG events", is_independent=False, is_binned=False, units="")
    nonres.values = NONRES["y"]
    
    rare = Variable("Number of rare BG events", is_independent=False, is_binned=False, units="")
    rare.values = RARE["y"]

    data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
    data.values = Data["y"]
    
    if sig == "METvBin_FBWH_SR" or sig == "METvBin_FBW_SR":
        unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=False)
        unc_totalbackground.values = list(zip(TotalBackground["dy"],TotalBackground["dy"]))
    else:       
        unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=True)
        unc_totalbackground.values = TotalBackground["dy"]
        
    if sig == "METvBin_FBWH_SR":
        unc_totalbackground.values[-2] = (-0.5,1.12)
        unc_totalbackground.values[-1] = (-0.43,1.09)
        
    if sig == "METvBin_FBW_SR":
        unc_totalbackground.values[-1] = (-1.03,1.70)
        
        
    #print(unc_totalbackground.values)

    unc_data = Uncertainty("Data uncertainty", is_symmetric=True)
    unc_data.values = Data["dy"]

    
    totalbackground.add_uncertainty(unc_totalbackground)
    data.add_uncertainty(unc_data)
    
    table.add_variable(xBins)
    table.add_variable(top)
    table.add_variable(nonres)
    table.add_variable(rare)
    table.add_variable(totalbackground)
    table.add_variable(sigWZ)
    table.add_variable(sigWH)
    table.add_variable(data)

    submission.add_table(table)


In [64]:
met_plot_btag("bTag_WHT_SRmaskedPredictions_Run2.root", "METvBin_FBWH_SR", "Figure 3b", \
              "SM background prediction vs. observation in the WH signal region", \
              "Data from Figure 3 (upper right), located on page 13.", ["N"], "Figure_003-b.pdf")

data_pred_ratio("bTag_WHT_SRmaskedPredictions_Run2.root", "METvBin_FBWH_SR", "Figure 3b ratios", \
 "SM background observation/prediction in the WH signal region", \
 "Data from Figure 3 (upper right), located on page 13.", ["N/N"], "Figure_003-b.pdf")

met_plot_btag("bTag_WT_SRmaskedPredictions_Run2.root", "METvBin_FBW_SR", "Figure 3c", \
              "SM background prediction vs. observation in the W signal region", \
              "Data from Figure 3 (lower left), located on page 13.", ["N"], "Figure_003-c.pdf")

data_pred_ratio("bTag_WT_SRmaskedPredictions_Run2.root", "METvBin_FBW_SR", "Figure 3c ratios", \
 "SM background observation/prediction in the W signal region", \
 "Data from Figure 3 (lower left), located on page 13.", ["N/N"], "Figure_003-c.pdf")

met_plot_btag("bTag_HT_SRmaskedPredictions_Run2.root", "METvBin_FBH_SR", "Figure 3d",  \
              "SM background prediction vs. observation in the H signal region", \
              "Data from Figure 3 (lower right), located on page 13.", ["N"], "Figure_003-d.pdf")

data_pred_ratio("bTag_HT_SRmaskedPredictions_Run2.root", "METvBin_FBH_SR", "Figure 3d ratios", \
 "SM background observation/prediction in the H signal region", \
 "Data from Figure 3 (lower right), located on page 13.", ["N/N"], "Figure_003-d.pdf")

central_unc.values=[(-0.15500510999956277, 0.15500510999956277), (-0.20275074568297088, 0.20275074568297088), (-0.24874661705558634, 0.24874661705558634), (-0.4328118362295033, 0.4328118362295033), (-0.3951672085862582, 0.3951672085862582), (-0.4840778115578834, 0.4840778115578834), (-0.42044278418002706, 0.9417918365632608), (-0.42317882949939245, 1.0727091259403205)]
central_unc.values=[(-0.06146955472438424, 0.06146955472438424), (-0.09787127376765409, 0.09787127376765409), (-0.0763001065679006, 0.0763001065679006), (-0.09949664854973363, 0.09949664854973363), (-0.11375292859162267, 0.11375292859162267), (-0.1349030838613576, 0.1349030838613576), (-0.13407059091112467, 0.13407059091112467), (-0.1709661865897351, 0.1709661865897351), (-0.2923777387742252, 0.482565199918624)]
central_unc.values=[0.04910028436919059, 0.04897512588680744, 0.06378268831004882, 0.08709659371178331, 0.1081435860756063, 0.10336133092795946, 0.1277268635500286, 0.1573668685414026, 0.3321338980842415]


In [65]:
def write_obs_contours(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = kw
    table.add_image(addImage)
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    #table.keywords["reactions"] = react
    
    reader = RootFileReader(inputfile)
    other=["TChiWH","TChiWW_WH"]
    if "TChiWH" in inputfile:
    #if(substring in inputfile for substring in other):
        print(inputfile)
        #print(substring)
        OBSWW = reader.read_graph("graph_smoothed_Obs_1")
        OBSPWW = reader.read_graph("graph_smoothed_ObsP_1")
        OBSMWW = reader.read_graph("graph_smoothed_ObsM_1")
    elif "TChiWW_WH" in inputfile:
        print(inputfile)
        #print(substring)
        OBSWW = reader.read_graph("graph_smoothed_Obs_1")
        OBSPWW = reader.read_graph("graph_smoothed_ObsP_1")
        OBSMWW = reader.read_graph("graph_smoothed_ObsM_1")
    else:
        OBSWW = reader.read_graph("graph_smoothed_Obs")
        OBSPWW = reader.read_graph("graph_smoothed_ObsP")
        OBSMWW = reader.read_graph("graph_smoothed_ObsM")
    
    # x-axis: Mass of NLSP
    mnlsp = Variable("m(NSLP)", is_independent=True, is_binned=False, units="GeV")
    mnlsp.values = OBSWW["x"] + OBSPWW["x"] + OBSMWW["x"]
    
    #y-axis: obs limit contours
    obsWW = Variable("max m(LSP) [GeV] excluded", is_independent=False, is_binned=False, units="")
    obsWW.values = OBSWW["y"] + ['~'] * len(OBSPWW["x"] + OBSMWW["x"])

    obsPWW = Variable("max m(LSP) [GeV] excluded +1 sigma", is_independent=False, is_binned=False, units="")
    obsPWW.values = ['~'] * len(OBSWW["x"]) + OBSPWW["y"] + ['~'] * len(OBSMWW["x"])

    obsMWW = Variable("max m(LSP) [GeV] excluded -1 sigma", is_independent=False, is_binned=False, units="")
    obsMWW.values = ['~'] * len(OBSWW["x"] + OBSPWW["x"]) + OBSMWW["y"] 
    
    print('obs values len: ',len(obsWW.values))
    
    table.add_variable(mnlsp)
    table.add_variable(obsWW)
    table.add_variable(obsPWW)
    table.add_variable(obsMWW)

    submission.add_table(table)

In [66]:
def write_exp_contours(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    #table.keywords["reactions"] = react

    if "TChiWZ" in inputfile:
        print(inputfile)
        #print(substring)
        EXPWW = reader.read_graph("graph_smoothed_Exp")
        EXPPWW = reader.read_graph("graph_smoothed_ExpP")
        EXPMWW = reader.read_graph("graph_smoothed_ExpM_2")
    elif "TChiWH" in inputfile:
        print(inputfile)
        #print(substring)
        EXPWW = reader.read_graph("graph_smoothed_Exp")
        EXPPWW = reader.read_graph("graph_smoothed_ExpP")
        EXPMWW = reader.read_graph("graph_smoothed_ExpM_1")
    elif "TChiWW_WH" in inputfile:
        print(inputfile)
        #print(substring)
        EXPWW = reader.read_graph("graph_smoothed_Exp")
        EXPPWW = reader.read_graph("graph_smoothed_ExpP") 
        EXPMWW = reader.read_graph("graph_smoothed_ExpM_1")
    else:
        EXPWW = reader.read_graph("graph_smoothed_Exp")
        EXPPWW = reader.read_graph("graph_smoothed_ExpP")
        EXPMWW = reader.read_graph("graph_smoothed_ExpM")
        
    # x-axis: Mass of NLSP
    mnlsp = Variable("m(NLSP)", is_independent=True, is_binned=False, units="GeV")
    mnlsp.values = EXPWW["x"] + EXPPWW["x"] + EXPMWW["x"]

    #y-axis: exp limit contours
    expWW = Variable("max m(LSP) [GeV] excluded", is_independent=False, is_binned=False, units="")
    expWW.values = EXPWW["y"] + ['~'] * len(EXPPWW["x"] + EXPMWW["x"])

    expPWW = Variable("max m(LSP) [GeV] excluded +1 sigma", is_independent=False, is_binned=False, units="")
    expPWW.values = ['~'] * len(EXPWW["x"]) + EXPPWW["y"] + ['~'] * len(EXPMWW["x"])

    expMWW = Variable("max m(LSP) [GeV] excluded -1 sigma", is_independent=False, is_binned=False, units="")
    expMWW.values = ['~'] * len(EXPWW["x"] + EXPPWW["x"]) + EXPMWW["y"] 

    table.add_variable(mnlsp)
    table.add_variable(expWW)
    table.add_variable(expPWW)
    table.add_variable(expMWW)


    submission.add_table(table)

In [67]:
def obs_heatmap(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    #table.keywords["reactions"] = react

    OBSXSEC = reader.read_hist_2d("hXsec_obs_corr")
#     print(len(OBSXSEC["z"]))
    zvalues = [i for i in OBSXSEC["z"] if i !=0]
    xvalues = [OBSXSEC["x_edges"][it] for it,i in enumerate(OBSXSEC["z"]) if i !=0]
    yvalues = [OBSXSEC["y_edges"][it] for it,i in enumerate(OBSXSEC["z"]) if i !=0]
    
    print('zvalues',len(zvalues))
    print('xvalues',len(xvalues))
    
    
    mnlsp = Variable("m(NLSP)", is_independent=True, is_binned=True, units="GeV")
    #mnlsp.values = OBSXSEC["x_edges"]
    mnlsp.values = xvalues
    print('mnlsp variable len: ', len(mnlsp.values))

    mlsp = Variable("m(LSP)", is_independent=True, is_binned=True, units="GeV")
    #mlsp.values = OBSXSEC["y_edges"]
    mlsp.values = yvalues

    obsxsec = Variable("Observed Limits", is_independent=False, is_binned=False, units="")
    #obsxsec.values = OBSXSEC["z"]
    obsxsec.values = zvalues
    #print('mnlsp variable len: ', obsxsec.values[:40])
    
    table.add_variable(mnlsp)
    table.add_variable(mlsp)
    table.add_variable(obsxsec)
    
    submission.add_table(table)

In [68]:
######## description for the table in the top section

obs_contour_lines="Observed exclusion limits assuming the approximate-NLO+NLL cross sections"
exp_contour_lines="Expected exclusion limits assuming the approximate-NLO+NLL cross sections"

WWobsXsec_desc="The 95% CL observed upper limits on the production cross sections for \
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$ assuming that each $\widetilde{\chi}^\pm_1$ \
decays to a W boson and $\widetilde{\chi}^0_1$"

WZobsXsec_desc="The 95% CL observed upper limits on the production cross sections for \
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ assuming that each $\widetilde{\chi}^\pm_1$ \
decays to a W boson and $\widetilde{\chi}^0_1$ and the $\widetilde{\chi}^0_2$ \
decays to a Z boson and $\widetilde{\chi}^0_1$"

WHobsXsec_desc="The 95% CL observed upper limits on the production cross sections for \
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ assuming that each $\widetilde{\chi}^\pm_1$ \
decays to a W boson and $\widetilde{\chi}^0_1$ and the $\widetilde{\chi}^0_2$ \
decays to a H boson and $\widetilde{\chi}^0_1$"

WWWZobsXsec_desc="The 95% CL observed upper limits on the production cross sections for mass-degenerate wino-like\
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$ and $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ as \
functions of the NLSP and LSP masses."

WWWHobsXsec_desc="The 95% CL observed upper limits on the production cross sections for mass-degenerate wino-like\
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$ and $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ as \
functions of the NLSP and LSP masses."

HinoobsXsec_desc="The 95% CL observed upper limits on the production cross sections for mass-degenerate higgsino-like\
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$, $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$, \
$\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_3$ and $\widetilde{\chi}^0_2$ $\widetilde{\chi}^0_3$  as \
functions of the NLSP and LSP masses."

In [69]:
#TChiWW
reactWW = ["P P --> $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^pm_1$", "$\widetilde{\chi}^pm_1$ --> W $\widetilde{\chi}^0_1$"]
write_obs_contours("afterCWR_TChiWW.root","Figure 4a Observed Lines",obs_contour_lines,\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

write_exp_contours("afterCWR_TChiWW.root","Figure 4a Expected Lines",exp_contour_lines,\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

obs_heatmap("afterCWR_TChiWW.root","Figure 4a Observed",WWobsXsec_desc,\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

obs values len:  170
zvalues 14977
xvalues 14977
mnlsp variable len:  14977


In [70]:
#TChiWZ
reactWZ = ["P P --> $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$", \
           "$\widetilde{\chi}^pm_1$ --> W $\widetilde{\chi}^0_1$", \
            "$\widetilde{\chi}^0_2$ --> Z $\widetilde{\chi}^0_1$"]
write_obs_contours("afterCWR_TChiWZ.root","Figure 4b Observed Lines",obs_contour_lines,\
                   "Data from Figure 4 (upper right), located on page 14.", ["Cross-section"], "Figure_004-b.pdf")

write_exp_contours("afterCWR_TChiWZ.root","Figure 4b Expected Lines",exp_contour_lines,\
                   "Data from Figure 4 (upper  right), located on page 14.", ["Cross-section"], "Figure_004-b.pdf")

obs_heatmap("afterCWR_TChiWZ.root","Figure 4b Observed",WZobsXsec_desc,\
                   "Data from Figure 4 (upper right), located on page 14.", ["CLS"], "Figure_004-b.pdf")

obs values len:  218
afterCWR_TChiWZ.root
zvalues 14977
xvalues 14977
mnlsp variable len:  14977


In [71]:
# #TChiWH
reactWH = ["P P --> $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$", \
           "$\widetilde{\chi}^pm_1$ --> W $\widetilde{\chi}^0_1$", \
            "$\widetilde{\chi}^0_2$ --> H $\widetilde{\chi}^0_1$"]
write_obs_contours("afterCWR_TChiWH.root","Figure 4c Observed Lines",obs_contour_lines,\
                   "Data from Figure 4 (lower), located on page 14.", ["Cross-section"], "Figure_004-c.pdf")

write_exp_contours("afterCWR_TChiWH.root","Figure 4c Expected Lines",exp_contour_lines,\
                   "Data from Figure 4 (lower), located on page 14.", ["Cross-section"], "Figure_004-c.pdf")

obs_heatmap("afterCWR_TChiWH.root","Figure 4c Observed",WHobsXsec_desc,\
                   "Data from Figure 4 (lower), located on page 14.", ["CLS"], "Figure_004-c.pdf")

afterCWR_TChiWH.root
obs values len:  266
afterCWR_TChiWH.root
zvalues 14585
xvalues 14585
mnlsp variable len:  14585


In [72]:
# TChiWW + WZ Wino combined Fig 5 left
reactWWWZ = ["P P --> $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ & $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$", \
           "$\widetilde{\chi}^pm_1$ --> W $\widetilde{\chi}^0_1$", \
            "$\widetilde{\chi}^0_2$ --> Z $\widetilde{\chi}^0_1$"]
write_obs_contours("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Observed Lines",obs_contour_lines,\
                   "Data from Figure 5 (left), located on page 14.", ["Cross-section"], "Figure_005-a.pdf")

write_exp_contours("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Expected Lines",exp_contour_lines,\
                   "Data from Figure 5 (left), located on page 14.", ["Cross-section"], "Figure_005-a.pdf")

# obs_heatmap("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Observed",WWWZobsXsec_desc,\
#                    "Data from Figure 5 (left), located on page 14.", ["CLS"], "Figure_005-a.pdf")

obs values len:  247


In [73]:
# TChiWW + WH Wino combined Fig 5 left
reactWWWH = ["P P --> $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^0_2$ & $\widetilde{\chi}^\pm_1$ $\widetilde{\chi}^\mp_1$", \
           "$\widetilde{\chi}^\pm_1$ --> W $\widetilde{\chi}^0_1$", \
            "$\widetilde{\chi}^0_2$ --> H $\widetilde{\chi}^0_1$"]
write_obs_contours("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Observed Lines",obs_contour_lines,\
                   "Data from Figure 5 (left), located on page 15.", ["Cross-section"], "Figure_005-a.pdf")

write_exp_contours("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Expected Lines",exp_contour_lines,\
                   "Data from Figure 5 (left), located on page 15.", ["Cross-section"], "Figure_005-a.pdf")

# obs_heatmap("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Observed",WWWHobsXsec_desc,\
#                    "Data from Figure 5 (left), located on page 15.", ["CLS"], "Figure_005-a.pdf")

test_TChiWW_WH_afterCWR.root
obs values len:  286
test_TChiWW_WH_afterCWR.root


In [74]:
# Hino combined Fig 5 right
reactHINO = ["P P --> $\widetilde{\chi}^\pm_1$ / $widetilde{\chi}^0_2$ / $\widetilde{\chi}^0_3$"]
write_obs_contours("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Observed Lines",obs_contour_lines,\
                   "Data from Figure 5 (right), located on page 15.", ["Cross-section"], "Figure_005-b.pdf")

write_exp_contours("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Expected Lines",exp_contour_lines,\
                   "Data from Figure 5 (right), located on page 15.", ["Cross-section"], "Figure_005-b.pdf")

obs_heatmap("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Observed",HinoobsXsec_desc,\
                   "Data from Figure 5 (right), located on page 15.", ["CLS"], "Figure_005-b.pdf")

obs values len:  67
zvalues 15312
xvalues 15312
mnlsp variable len:  15312


In [75]:
submission.create_files("Test2",remove_old=True)

Full-size PNG file Test2/Figure_003-b.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Thumbnail PNG file Test2/thumb_Figure_003-b.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Full-size PNG file Test2/Figure_003-c.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Thumbnail PNG file Test2/thumb_Figure_003-c.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Full-size PNG file Test2/Figure_003-d.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)      